### User input

In [ ]:
decimal = '.'                                                            # decimal of the input file
sep = ';'                                                                # separator of the input file
input_dir = 'C:/Users/cace0002/AbspectroscoPY/results/df_nonan_sw.csv'   # input directory
output = 'C:/Users/cace0002/AbspectroscoPY/results/'                     # output directory
dateheadername = 'Timestamp'                                             # header of the date  
header = 0                                                               # header row number
sample_name = 'sw'                                                       # name of the sample

#To plot the duplicates, the user can modify:
col_sel = '220 nm'                                                       # column to plot
timestart = '2018-11-12 16:04:00'                                        # starting date
timeend = '2018-12-03 20:44:00'                                          # ending date
title =  'duplicates_'                                                   # title of the exported figure
fig_format = '.tiff'                                                     # format of the exported figure
dpi = 300                                                                # resolution of the exported figure

### Start environment and import data

In [ ]:
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0)
df.index = pd.to_datetime(df.index)      # make sure time column (here index) is using time format
df

### dup_check

In [ ]:
def dup_check(df_in,
             dateheadername):
    '''
    function to check and plot duplicates
    :argument df_in: dataframe in input
    :argument dateheadername: name of the date column
    :return: two dataframes with duplicates by dateheadername and by all columns
    '''    
    df_out1 = df_in[df_in.index.duplicated(keep=False)] # check for duplicates by dateheadername     
    df_out2 = df_in[df_in.duplicated()]                 # check for duplicates by all the columns and report them once 
    return(df_out1, df_out2)     

In [ ]:
df_dup, df_dup_all = dup_check(df, dateheadername)
df_dup_all.to_csv(output + '_duplicates.csv', sep= sep, decimal = decimal, index=False) # export all duplicates

### Plot duplicates

In [ ]:
abspy.makeaplot(df_dup_all, output, col_sel, timestart, timeend, sample_name, title) # Run twice if the plot looks too small

### Drop duplicates

In [ ]:
df_nodupall = df.drop_duplicates(subset = None, keep = "first", inplace = False)                               # drop second duplicate by all columns
df_nodupall.to_csv(output + 'df_nodup_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the dataframe
#df_nodupall